# objectif du notebook
- création les jeux de données à annoter manuellement sur excel
    - distribution aléatoire entre N annotateurs
- lecture des jeux de données annotés sur excel
- creation d'un dictionnaire pour les données de références (DR): [clé=nom_theme, valeur=list(code_theme)]
    - ex: 'livre-jeu': ['YBCS2', 'YBG']
        ici: pour le même nom_theme, chaque evaluateur a donné un code Thema différent

# création les jeux de données à annoter manuellement sur excel

In [3]:
import os 
import random
import itertools
import uuid
import pandas as pd
import sklearn.metrics
import sys
sys.path.append('../')
sys.path.append('../andre')
import andre.utils as utils
import copy

## chargement des données et prétraitement

In [4]:
ecole_des_loisirs = ['âne',
 'animaux des champs et des bois',
 'animaux divers',
 'animaux domestiques',
 'araignée',
 'autruche, émeu',
 'baleine',
 'belette',
 'bison',
 'blaireau',
 'caméléon',
 'canard, oie',
 'castor',
 'cervidés',
 'chameau, dromadaire',
 'chamois',
 'chat',
 'chauve souris',
 'cheval',
 'chien',
 'cochon',
 'coyote, chacal',
 'crabe',
 'crocodile, alligator',
 'dauphin',
 'des animaux et des hommes',
 'dindon',
 'dinosaure (animal)',
 'écureuil',
 'éléphant',
 'escargot',
 'félins divers',
 'girafe',
 'gnou',
 'grenouille, crapaud, têtard',
 'hamster, cobaye',
 'hérisson, porc-épic',
 'hibou',
 'hippocampe',
 'hippopotame',
 'insectes divers',
 'kangourou',
 'koala',
 'lamantin',
 'lapin, lièvre',
 'lézard',
 'lion',
 'loir, lérot',
 'loup',
 'loutre',
 'lynx',
 'mammouth',
 'marmotte',
 'morse',
 'mouton, chèvre, bouc',
 'oiseaux divers',
 'opossum',
 'ornithorynque',
 'orque',
 'ours',
 'panda',
 'papillon',
 'paresseux (animal)',
 'phoque',
 'pieuvre, poulpe',
 'pingouin, manchot',
 'poissons divers',
 'poule, poussin, coq',
 'raton laveur',
 'renard',
 'requin',
 'rhinocéros',
 'sanglier',
 'serpent',
 'singe, chimpanzé, gorille',
 'souris, mulot, rat, campagnol',
 'suricate, mangouste',
 'tatou',
 'taupe',
 'tigre',
 'tortue',
 'vache, veau, taureau',
 'ver de terre, lombric',
 'yéti',
 'zèbre',
 'art pariétal, peinture rupestre',
 'art traditionnel, arts premiers',
 'cinéma',
 'cirque',
 'clown',
 "commedia dell'arte",
 'créativité, création artistique',
 'danse',
 'funambule',
 'guignol',
 'Littérature, livre, écrivain',
 'marionnette',
 'mime',
 'musique',
 'opéra',
 'peinture',
 'photographie',
 'photographie (livre de)',
 'sculpture',
 'street-art',
 'théâtre (activité)',
 'dent, dent de lait',
 'main',
 'schéma corporel',
 'squelette',
 'hygiène, propreté',
 'pipi, caca, toilettes',
 'pot',
 'poux',
 'beauté, laideur',
 'coquetterie',
 'imagerie du corps',
 'autisme',
 'corps et ses différences (le)',
 'handicap',
 'lunettes',
 'grossesse, avortement',
 'naissance, maternité',
 'accident',
 'alcoolisme',
 'anorexie',
 'appétit',
 'bobo',
 'boulimie',
 'drogue, toxicomanie',
 'épidémie',
 'hospitalisation',
 'maladie, docteur',
 'pipi au lit, énurésie',
 'santé',
 'sida',
 'homosexualité',
 'puberté, transformation du corps',
 'sexualité',
 'dormir, coucher',
 'peur du noir, cauchemar',
 'rêve',
 'les cinq sens',
 'prévention (le corps)',
 'repère spatio temporel',
 'taille',
 "apprentissage de l'autonomie",
 'apprentissage de la vie',
 'premiers apprentissages',
 'bébé',
 'crèche',
 'tétine, biberon',
 'apprentissage des langues',
 'classe de nature',
 'collège, lycée, professeur',
 'école buissonnière',
 'école, instituteur',
 'pensionnat',
 'rentrée des classes',
 'sortie scolaire',
 'université',
 'mort, deuil',
 'prise de responsabilité',
 'temps (qui passe)',
 'vieillesse',
 'généalogie',
 'recherche de ses origines',
 'bêtises, désobéissance',
 'conflit parents, enfants',
 'fratrie',
 'fugue',
 'grand-mère',
 'grand-père',
 'grands-parents',
 'homoparentalité',
 'jumeaux, jumelles',
 'naissance, nouveau bébé dans la fratrie',
 'relation avec la mère',
 'relation avec le père',
 'relation enfant, adulte',
 'sœurs et frères : aîné, cadet',
 'déménagement',
 'désordre',
 'vie quotidienne',
 'adoption',
 'dispute',
 'divorce, séparation',
 'en famille',
 'enfant abandonné, abandon',
 'enfant maltraité',
 "famille d'accueil",
 'famille éclatée',
 'famille monoparentale',
 'famille recomposée, remariage',
 'la famille',
 'mariage',
 'orphelin, orpheline',
 'secrets de famille',
 'vacances en famille',
 'aventure et mystère',
 "cape et d'épée",
 'histoire policière',
 'récit de voyage',
 "roman d'aventure",
 'conte détourné',
 'conte détourné, le bal des 12 princesses',
 'conte détourné, barbe bleue',
 'conte détourné, blanche-neige',
 "conte détourné, boucle d'or et les 3 ours",
 'conte détourné, cendrillon',
 'conte détourné, la belle au bois dormant',
 'conte détourné, la princesse au petit pois',
 'conte détourné, le loup et les 7 cabris',
 'conte détourné, le petit chaperon rouge',
 'conte détourné, le petit poucet',
 'conte détourné, les 3 petits cochons',
 'conte détourné, les fées',
 'contes du monde entier',
 'contes et recueil de contes',
 'fables',
 'classique abrégé',
 'classique intégral',
 "classiques du livre d'images",
 'fantastique',
 'fantasy',
 'science-fiction',
 'fables, fabliaux',
 'Littérature du Moyen Âge',
 'roman de la table ronde',
 'cow-boy',
 'super-héros',
 'ange',
 'chevalier',
 'diable',
 'dragon',
 'fantôme',
 'fée',
 'géant',
 'génie',
 'licorne',
 'lutin, troll, elfe, gnome',
 'magicien, magie',
 'monstre',
 'nain',
 'ogre',
 'personnage de conte (divers)',
 'prince, princesse',
 'roi, reine',
 'sirène',
 'sorcier, sorcière',
 'vampire',
 'sketch',
 'théâtre classique',
 'théâtre contemporain',
 'théâtre, pièce de théâtre',
 'atlas',
 'autobiographie',
 'aventure',
 'bande dessinée',
 'biographie',
 'comptine',
 'conte',
 'énigmes',
 'épouvante',
 'extraterrestre',
 'humour',
 'journal',
 'lettres, correspondance',
 'livre dont on est le héros',
 'Livre ukrainien',
 'Livre-jeu',
 'MOOC',
 'mythologie',
 'parodie, pastiche',
 'philosophie',
 'poésie',
 'proverbe',
 "recueil d'histoires",
 'recueil de nouvelles',
 'rime',
 "roman de cap et d'épée",
 'texte fondateur',
 'Afrique (continent africain)',
 'Afrique du nord',
 'Afrique du sud',
 'Algérie',
 'Egypte',
 'Kenya',
 'Libye',
 'Madagascar',
 'Maghreb',
 'Mali',
 'Maroc',
 'Seychelles (îles des)',
 'Soudan',
 'Tanzanie',
 'Tunisie',
 'Zanzibar',
 'Antilles, Caraïbes',
 'Costa-Rica',
 'Cuba',
 'Guatemala',
 'Haïti',
 'Mexique',
 'Porto-Rico',
 'Alaska',
 'Canada',
 'Etats-Unis',
 'Argentine',
 'Bolivie',
 'Brésil',
 'Chili',
 'Colombie',
 'Equateur',
 'Pérou',
 'Vénézuela',
 'Afghanistan',
 'Arabie',
 'Arménie',
 'Asie (en général)',
 'Bahrein',
 'Bangladesh',
 'Birmanie',
 'Bornéo',
 'Cambodge',
 'Chine',
 'Corée',
 'Emirats',
 'Extrème-Orient',
 'Inde',
 'Indonésie',
 'Irak',
 'Iran',
 'Israël, palestine',
 'Japon',
 'Jordanie',
 'Kirghizistan',
 'Koweït',
 'Laos',
 'Liban',
 'Malaisie',
 'Mongolie',
 'Moyen-Orient',
 'Népal',
 'Oman',
 'Pakistan',
 'Philippines',
 'Sri lanka',
 'Syrie',
 'Taïwan',
 'Thaïlande',
 'Tibet',
 'Viêtnam',
 'Yémen',
 'Géorgie',
 'Ouzbékistan',
 'Russie',
 'Turquie',
 'URSS',
 'Esquimau, Inuit',
 'Gaulois',
 'Indien',
 "Indiens d'Amérique",
 'Mayas',
 'Monde arabe',
 'Touareg',
 'Vikings',
 'Allemagne',
 'Autriche',
 'Balkans',
 'Belgique',
 'Danemark',
 'Espagne',
 'Europe',
 'Finlande',
 'France',
 'Grande-Bretagne',
 'Grèce',
 'Hongrie',
 'Islande',
 'Italie',
 'Norvège',
 'Pays nordiques',
 'Pays-Bas',
 'Pologne',
 'Portugal',
 'Roumanie',
 'Suède',
 'Suisse',
 'Tchécoslovaquie',
 'Ukraine',
 'Amazonie',
 'banquise',
 'désert',
 'forêt',
 'île',
 'jungle, forêt lointaine',
 'montagne',
 'rivière, fleuve, étang',
 'Sahara',
 'volcan',
 'Australie',
 'Nouvelle-Calédonie',
 'Nouvelle-Guinée',
 'Nouvelle-Zélande',
 'Polynésie',
 'Arctique, Antarctique',
 'camp de concentration',
 'Commune de paris',
 'conquête',
 'déportation',
 'entre deux guerres',
 'fascisme',
 'génocide arménien',
 'génocide khmère',
 'guerre, conflit',
 'Hiroshima',
 'mafia',
 'nazisme',
 'Première Guerre mondiale',
 'révolution',
 'Révolution française',
 'Seconde Guerre mondiale',
 'terrorisme, guerre civile',
 'Antiquité',
 "Antiquité : l'Egypte",
 'Antiquité : la Gaule',
 'Antiquité : la Grèce',
 'Antiquité : Rome',
 'Gaule (la)',
 "Histoire (l'an 1000)",
 'Histoire : 13e siècle',
 'Histoire : 15e siècle',
 'Histoire : 16e siècle',
 'Histoire : 17e siècle',
 'Histoire : 18e siècle',
 'Histoire : 19e siècle',
 'Histoire : 20e siècle',
 'Histoire : 21e siècle',
 'Histoire : 5e siècle',
 'Histoire : Moyen Âge',
 'Préhistoire',
 'Martin Luther King',
 'château, château fort',
 'Cro-Magnon (Homme de)',
 'dinosaure',
 'Histoire du monde',
 'Mésopotamie',
 'personnage historique',
 'pionniers',
 'bruits et cris',
 'chant, chanson',
 'contraire',
 'couleurs',
 'étymologie',
 'histoire sans texte',
 'images pour jouer',
 'imagier',
 'langage, jeux de mots',
 'livre à compter, chiffres',
 'métamorphose (image)',
 'balle, ballon',
 'cache-cache',
 'cerf-volant',
 'course',
 'déguisement',
 'doudou',
 'jeu',
 'jouets',
 'masque',
 'ours en peluche',
 'poupée',
 'robot',
 'abécédaire',
 'almanach',
 'coloriage, livre à colorier',
 'jeu de rôle',
 'jeu vidéo',
 'les échecs',
 'livre animé',
 'livre de bain',
 'livre de naissance',
 'livre puzzle',
 'chef, meneur, leader',
 'courageux',
 'cupide',
 'curiosité',
 'égoïste',
 'fantaisie, fantaisiste',
 'farceur, farceuse',
 'ingénieux, débrouillard',
 'paresseux',
 'rabat-joie, grincheux',
 'responsable',
 'rêveur, poète',
 'rusé, astucieux',
 'vantard',
 'ami imaginaire',
 'amitié',
 'amour',
 'caprices, entêtement',
 'colère',
 'éco-anxiété',
 'émotion',
 'ennui',
 'gratitude',
 'harcèlement',
 'honte, gêne',
 'humeur',
 'jalousie',
 'méchanceté, cruauté',
 'mensonge',
 'moquerie',
 'nostalgie',
 'peur',
 'regard des autres',
 'rivalité, bagarre, dispute',
 "sentiment d'injustice",
 'sentiment de culpabilité',
 'tendresse, baiser, câlin',
 'timidité',
 'tristesse, chagrin',
 'vengeance',
 'affirmation de soi',
 'genre',
 'imagination, fabulation',
 'initiation, épreuve',
 "le complexe d'Oedipe",
 'portrait',
 "recherche d'absolu",
 "recherche d'identité",
 'secret',
 'souvenir, mémoire',
 'télépathie',
 'thérapie',
 'transgenre',
 'transmission du savoir',
 'hibernation',
 'instinct, education',
 'nourriture, qui mange qui ?',
 'reproduction animale',
 'trace',
 'zoologie',
 'arbre',
 'champignons',
 'fleur',
 'fruit',
 'légume',
 'catastrophe naturelle',
 'intempérie',
 'météo',
 'neige',
 'nuage',
 'pluie, orage',
 'soleil',
 'vent',
 'automne',
 'cycle des saisons',
 'été',
 'heure',
 'hiver',
 'printemps',
 'semaine',
 'biologie',
 'déchets',
 'eau',
 'écologie, environnement',
 'espace, air',
 'feu',
 'métamorphose (biologie)',
 'naturaliste',
 'nuit (dans la)',
 'pierre, caillou, minéral',
 'planète terre',
 "protection de l'environnement",
 'reproduction végétale',
 'vie sous-marine',
 'vie souterraine',
 'astronomie',
 'aventure aérienne et spatiale',
 'ciel',
 'conquête spatiale',
 'cosmonaute, spationaute',
 'étoile',
 'lune',
 'classification des espèces',
 'imprimerie',
 'savant(e), femme/homme de science, inventeur',
 'technologie',
 'avion',
 'bateau',
 'camion, voiture',
 'code de la route',
 'engins divers',
 'métro',
 'montgolfière',
 'moyen de transport',
 'port',
 'train',
 'archéologie',
 'architecture',
 'clonage humain',
 'informatique, internet',
 'physique, chimie',
 'sismologie',
 'spéléologie',
 'transformation des produits naturels',
 'internet',
 'médias',
 'publicité',
 'réseaux sociaux',
 'télévision',
 'conditions sociales',
 'pauvreté, précarité',
 'travail (conditions de)',
 'sectes',
 'sorcellerie',
 'Bohémien, Gitan',
 'bonbon',
 'chapeau',
 'cuisine',
 'culture, tradition',
 'gourmandise',
 'nourriture',
 'vêtement',
 'comportements sociaux',
 'anniversaire et fête',
 'carnaval',
 'fête foraine',
 'galette des rois',
 'halloween',
 'Noël',
 'Noël (sapin de)',
 'Pâques',
 'père noël',
 "pique-nique, goûter d'enfants",
 'bricolage',
 'camping',
 'jeux de rôles',
 'jeux vidéo',
 'livre, lecture',
 'manège',
 'scoutisme',
 'esclavage',
 'exil',
 'immigration, migration',
 'intégration',
 'réfugié, sans-papier',
 'apartheid',
 'communisme',
 'conscience politique',
 'démocratie',
 'dictature',
 'Khmers rouges',
 'totalitarisme',
 'arche de noé',
 'bible',
 'foi',
 'intégrisme',
 'juive (culture)',
 'religions, croyances',
 'arts martiaux',
 'boxe',
 'équitation',
 'exploit sportif',
 'football',
 'Jeux olympiques',
 'natation',
 'patin à glace',
 'roller',
 'rugby',
 'ski',
 'sport',
 'surf',
 'tennis',
 'vélo',
 'yoga',
 'inceste',
 'interdit',
 'justice',
 'maltraitance',
 'prison',
 'violence',
 'violence sexuelle',
 'voleur',
 'argent (système économique)',
 'argent de poche',
 'baby-sitting',
 'boulanger, pâtissier',
 'casting',
 'chômage',
 'coiffeur, coiffure',
 'détective',
 'docteur, médecin',
 'écrivain, écriture',
 'explorateur',
 'facteur',
 'journalisme',
 'juge',
 'métiers divers',
 'péniche, marinier',
 'petits boulots',
 'pompier',
 'travail',
 'travail des enfants',
 'vétérinaire',
 'colonie de vacances, centre de loisirs',
 'tour du monde',
 'vacances',
 "vacances à l'étranger",
 'vacances à la campagne',
 'vacances à la mer',
 'vacances à la montagne',
 'voyage',
 'argent',
 'citoyenneté',
 'communication',
 'confinement',
 'piscine',
 'pouvoir',
 'transhumanisme',
 'campagne',
 'ferme',
 'pêche',
 'retour à la nature',
 'vie rurale',
 'mer',
 'naufrage',
 'pirate, corsaire',
 'plage, bord de mer',
 'conditions de vie dans les villes',
 'jardin public',
 'magasin',
 'marché',
 'musée',
 'poste',
 'rue',
 'urbanisme',
 'ville (la)',
 'zoo',
 'cabane',
 'jardin',
 'lieux et territoires, campagne',
 'maison, habitation']

In [5]:
ecole_des_loisirs_pretraite = [word.lower() for word in ecole_des_loisirs]

In [6]:
ecole_des_loisirs_pretraite

['âne',
 'animaux des champs et des bois',
 'animaux divers',
 'animaux domestiques',
 'araignée',
 'autruche, émeu',
 'baleine',
 'belette',
 'bison',
 'blaireau',
 'caméléon',
 'canard, oie',
 'castor',
 'cervidés',
 'chameau, dromadaire',
 'chamois',
 'chat',
 'chauve souris',
 'cheval',
 'chien',
 'cochon',
 'coyote, chacal',
 'crabe',
 'crocodile, alligator',
 'dauphin',
 'des animaux et des hommes',
 'dindon',
 'dinosaure (animal)',
 'écureuil',
 'éléphant',
 'escargot',
 'félins divers',
 'girafe',
 'gnou',
 'grenouille, crapaud, têtard',
 'hamster, cobaye',
 'hérisson, porc-épic',
 'hibou',
 'hippocampe',
 'hippopotame',
 'insectes divers',
 'kangourou',
 'koala',
 'lamantin',
 'lapin, lièvre',
 'lézard',
 'lion',
 'loir, lérot',
 'loup',
 'loutre',
 'lynx',
 'mammouth',
 'marmotte',
 'morse',
 'mouton, chèvre, bouc',
 'oiseaux divers',
 'opossum',
 'ornithorynque',
 'orque',
 'ours',
 'panda',
 'papillon',
 'paresseux (animal)',
 'phoque',
 'pieuvre, poulpe',
 'pingouin, manch

In [7]:
len(ecole_des_loisirs_pretraite)

744

In [8]:
ricochet = ["Abandon/Sentiment d'abandon",
 'Abécédaire/Alphabet',
 'Abeille',
 'Aborigène',
 'Absence',
 'Absurde',
 'Abus sexuel',
 'Accident',
 'Activités manuelles',
 "Activités/Cahier d'activités",
 "Adaptation (d'œuvre littéraire)",
 'Adolescence',
 'Adoption',
 'Adultère',
 'Affection/Tendresse',
 'Affirmation de soi',
 'Afghanistan',
 'Afrique',
 'Afrique du Nord',
 'Afrique du Sud',
 'Aide humanitaire',
 'Alaska',
 'Album',
 'Album sans texte',
 'Alcool/Alcoolisme',
 'Algérie',
 'Alimentation/Goût',
 'Allemagne',
 'Alphabétisation',
 'Alpinisme',
 'Amérique du Nord',
 'Amérique latine',
 'Amitié',
 'Amnésie',
 'Amour',
 'Analyse de texte',
 'Ane',
 'Ange',
 'Animaux',
 'Anniversaire/Fête',
 'Anorexie/Boulimie',
 'Antilles/Caraïbes',
 'Antiquité (Gaule)',
 'Antiquité (Grèce)',
 'Antiquité (Rome)',
 'Antisémitisme',
 'Apartheid',
 'Apprentissage',
 'Apprentissage de la vie',
 'Araignée',
 'Arbre',
 'Archéologie',
 'Architecture',
 'Argent/Argent de poche',
 'Argentine',
 'Arménie',
 'Art',
 'Art brut',
 'Art urbain',
 'Asie',
 'Astrologie',
 'Astronomie',
 "Atelier d'écriture",
 'Australie',
 'Autisme',
 'Autobiographie',
 'Automne',
 'Automobile/Voiture',
 'Autriche',
 'Autruche',
 'Aventure',
 'Avion/Aviation',
 'Bain/Toilette',
 'Bande dessinée',
 'Bande dessinée (Histoire)',
 'Bandit/Brigand',
 'Banlieue',
 'Bateau/Voilier',
 'Beauté/Laideur',
 'Bébé',
 'Belgique',
 'Berceuse',
 'Bestiaire',
 'Bêtise/Désobéissance',
 'Bhoutan',
 'Bible',
 'Bibliographie/Sélection de livre',
 'Bibliothèque',
 'Bibliothèque jeunesse',
 'Bicyclette/Vélo',
 'Bien-être animal',
 'Bijou/Diamant',
 'Biographie',
 'Bise/Bisou/Baiser',
 'Bison',
 'Blaireau',
 'Bolivie',
 'Bonheur',
 'Botanique',
 'Bouddhisme',
 'Boxe',
 'Brésil',
 'Bretagne',
 'Bruits/Cris',
 'Bulgarie',
 'Burundi',
 'Caillou/Rocher',
 'Calligraphie',
 'Cambodge',
 'Cameroun',
 'Campagne/Ferme',
 'Camping',
 'Canada',
 'Canard/Oie',
 'Caprice/Entêtement',
 'Carnaval',
 'Castor',
 'Catastrophe naturelle',
 'Catastrophe nucléaire',
 'Cécité',
 'Célébrité',
 'Censure',
 'Cerf-volant',
 'Cerf/Biche',
 'Chameau/Dromadaire',
 'Champignon',
 'Chanson/Chant',
 'Chapeau',
 'Chat',
 'Château',
 'Chaud/Froid',
 'Chauve-souris',
 'Chenille',
 'Cheval/Equitation',
 'Chevalier/Chevalerie',
 'Chien',
 'Chiffre/Nombre/Livre à compter',
 'Chimie',
 'Chine',
 'Chocolat',
 'Chômage',
 'Chouette/Hibou',
 'Ciel',
 'Cinéma/Vidéo',
 'Cinq sens',
 'Cirque',
 'Citation',
 'Citoyenneté/Civisme',
 'Clown',
 'Coccinelle',
 'Cochon/Sanglier',
 'Coiffeur',
 'Colère',
 'Collection/Collectionneur',
 'Collège',
 'Colombie',
 'Colonie de vacances',
 'Colonisation/Colonie',
 'Coloriage',
 'Comédie musicale',
 'Communication',
 'Compétition',
 'Comptine',
 'Concentration',
 'Condamnation à mort',
 'Condition féminine',
 'Condition sociale',
 'Conformisme',
 'Congo',
 "Conquête de l'Ouest américain",
 'Consommation',
 'Conte',
 'Conte (adaptation)',
 'Contraire',
 'Corée',
 'Corps',
 'Correspondance',
 'Corrida',
 'Corse',
 'Couleur',
 'Courage',
 'Crocodile/Alligator',
 'Cuba',
 'Cuisine/Recette',
 'Culpabilité',
 'Curiosité',
 'Cygne',
 'Danemark',
 'Danse/Ballet',
 'Dauphin/Baleine',
 'Débarquement',
 'Découverte',
 'Déguisement',
 'Délinquance',
 'Déménagement',
 'Dent',
 'Dépression',
 'Désert',
 'Désespoir',
 'Design',
 'Dessin',
 'Dessin animé',
 'Détente/Relaxation',
 'Développement durable',
 'Devinette',
 'Diable',
 'Dictature',
 'Différence',
 'Dinosaure',
 'Disparition',
 'Documentaire',
 'Dormir/Coucher',
 'Double',
 'Doudou/Peluche',
 'Dragon',
 'Drogue',
 'Droit',
 "Droits de l'enfant",
 "Droits de l'homme",
 'Dyslexie/Trouble de la lecture',
 'Dystopie',
 'Eau',
 'Ecole',
 'Ecole maternelle',
 'Ecole primaire',
 'Ecologie',
 'Economie/Finance',
 'Ecosse',
 'Ecriture',
 'Ecureuil',
 'Edition',
 'Edition - Petite Edition',
 'Education',
 'Egalité',
 'Egoïsme',
 'Egypte',
 'Egypte ancienne',
 'Elan',
 'Electricité',
 'Eléphant',
 'Emotions',
 'Encyclopédie/Dictionnaire',
 'Energie',
 'Enfance',
 'Enfant des rues',
 'Enfant sauvage',
 'Enigme',
 'Enlèvement/Kidnapping',
 'Ennui',
 'Enquête',
 'Entraide/Solidarité',
 'Environnement',
 'Epreuve/Initiation',
 'Équateur',
 'Escargot',
 'Esclavage',
 'Espace/vaisseaux spatiaux',
 'Espagne',
 'Espionnage/Espion',
 'Espoir',
 'Etats-Unis',
 'Eté',
 'Ethiopie',
 'Ethique/Morale',
 'Etoile',
 'Europe',
 "Europe de l'Est",
 'Excision',
 'Exclusion',
 'Exil',
 'Exploit',
 'Exposition',
 'Extraterrestre',
 'Fable',
 'Famille',
 "Famille d'accueil",
 'Famille recomposée',
 'Fantaisie',
 'Fantastique',
 'Fantôme',
 'Fascisme',
 'Fée',
 'Femme/Homme',
 'Fête foraine',
 'Feu/Incendie',
 'Fille/Garçon',
 'Finlande',
 'Fleur',
 'Football',
 'Forêt',
 'Forme',
 'Fourmi',
 'France',
 'Fruit',
 'Fugue',
 'Généalogie/Arbre généalogique',
 'Générosité',
 'Géographie/Atlas',
 'Girafe',
 'Gitan/Bohémien',
 'Gourmandise',
 'Grand Nord',
 'Grand-Mère',
 'Grand-Père',
 'Grand/Petit',
 'Grande-Bretagne',
 'Grèce',
 'Grenouille/Crapaud',
 'Grève',
 'Grimace',
 'Gros/Maigre',
 'Guatemala',
 'Guerre/Conflit',
 'Guide/Répertoire',
 'Guyane',
 'Habitation/Maison',
 'Halloween',
 "Hamster/Cochon d'Inde",
 'Handicap',
 'Harcèlement',
 'Hérisson',
 'Héros',
 'Heure',
 'Hippopotame',
 'Histoire',
 'Histoire (Moyen-Age)',
 'Histoire (Première Guerre mondiale)',
 'Histoire (Révolution française)',
 'Histoire (Seconde Guerre mondiale)',
 'Histoire (XIX° siècle)',
 'Histoire (XVIII° siècle)',
 'Histoire (XVII° siècle)',
 'Histoire (XVI° siècle)',
 'Histoire (XV° siècle)',
 'Histoire (XXI° siècle)',
 'Histoire (XX° siècle)',
 "Histoire de l'art",
 'Hiver',
 'Holocauste',
 'Homosexualité',
 'Hongrie',
 'Honte/Gêne',
 'Hôpital',
 'Humour',
 'Hygiène/Propreté',
 'Identité',
 'Illettrisme',
 'Illustration',
 'Imagier',
 'Imagination/Fabulation',
 'Immigration/Emigration',
 'Immortalité',
 'Imprimerie',
 'Inca/Aztèque',
 'Inde',
 'Indien',
 'Individualisme',
 'Indonésie',
 'Inégalité/Injustice',
 'Information/Actualité',
 'Insecte',
 'Instrument de musique',
 'Intégration',
 'Interculturalité',
 'Internet',
 'Inuit',
 'Irak',
 'Iran',
 'Irlande',
 'Islande',
 'Israël',
 'Italie',
 'Jalousie',
 'Jamaïque',
 'Japon',
 'Jardin',
 'Jardinage/Potager',
 'Jazz',
 "Jeu d'optique / Illusion d'optique",
 'Jeu de doigts',
 'Jeu de mots',
 'Jeu vidéo',
 'Jeu/Jouet',
 'Jeux Olympiques',
 'Journal intime',
 'Journalisme',
 'Judaïsme',
 'Jumeaux/Jumelles',
 'Justice',
 'Kamishibaï',
 'Kangourou',
 'Koala',
 'Kosovo',
 'La Réunion',
 "Lac/Cours d'eau",
 'Langue des signes',
 'Langue étrangère (apprentissage)',
 'Langue/Langage',
 'Lapin/Lièvre',
 "Lecture de l'image",
 'Légende arthurienne',
 'Légendes/Merveilleux',
 'Légume',
 'Lemming',
 'Léopard',
 'Lever/Réveil',
 'Liban',
 'Liberté',
 'Librairie',
 'Licorne',
 'Lion',
 'Liste',
 'Littérature',
 'Littérature de jeunesse',
 'Lituanie',
 'Livre animé',
 'Livre-jeu',
 'Livre/Lecture',
 'Londres',
 'Loup',
 'Loup-garou',
 'Lumière',
 'Lune',
 'Lutin',
 'Luxembourg',
 'Lycée',
 'Madagascar',
 'Magasin/Commerce',
 'Magie/Magicien',
 'Maladie',
 'Malaisie',
 'Mali',
 'Maltraitance',
 'Mammouth',
 'Manège',
 'Manga',
 'Manipulation',
 'Manipulation génétique/Clonage',
 'Mariage',
 'Mariage forcé',
 'Marionnette/Guignol',
 'Maroc',
 'Masque',
 'Mathématique',
 'Méchanceté/Cruauté',
 'Médecin/Docteur',
 'Médicament',
 'Mélancolie',
 'Mensonge/Vérité',
 'Menstruation/Règles',
 'Mer/Océan',
 'Métamorphose',
 'Météo/Climat',
 'Métier',
 'Métro',
 'Mexique',
 'Mille et une nuits',
 'Mine/Mineur',
 'Misère/Pauvreté',
 'Mode',
 'Monde arabe',
 'Mongolie',
 'Monstre',
 'Montagne',
 'Moquerie',
 'Mort/Deuil',
 'Mouche',
 'Mouette',
 'Moulin',
 'Mouton/Chèvre',
 'Moyen-Orient',
 'Musée',
 'Musique',
 'Mystère',
 'Mythologie',
 'Naissance/maternité',
 'Napoléon',
 'Narrateur/Narration',
 'Nature',
 'Naufrage',
 'Neige',
 'Népal',
 'New York',
 'Noël',
 'Normandie',
 'Norvège',
 'Nourrice/Baby-sitter',
 'Nourriture',
 'Nouvelle-Calédonie',
 'Nouvelle-Zélande',
 'Nouvelles technologies',
 'Nuit',
 'Obésité',
 'Océanie',
 'Odorat',
 'Oeuf',
 'Ogre/Géant',
 'Oiseau',
 'Opéra',
 'Ordinateur/Informatique',
 'Origami',
 'Orphelin/Orphelinat',
 'Ouïe',
 'Ours',
 'Paix/Pacifisme',
 'Pakistan',
 'Palestine',
 'Panda',
 'Papillon',
 'Pâques',
 'Paresse',
 'Parfum',
 'Paris',
 'Parodie',
 'Partage',
 'Paternité',
 'Patience/Impatience',
 'Pays-Bas',
 'Paysage',
 'Pêche/Pêcheur',
 'Pédagogie',
 'Pédophilie',
 'peintre',
 'Peinture',
 'Père Noël',
 'Pérou',
 'Perse',
 'Persévérance',
 'Petite enfance',
 'Peur',
 'Peur du noir/Cauchemar',
 'Philosophie',
 'Phoque/Morse',
 'Photographie',
 'Pingouin/Manchot',
 'Pique-nique',
 'Pirate/Corsaire',
 'Piscine',
 'Planète',
 'Plante',
 'Pluie/Orage',
 'Poésie',
 'Poison',
 'Poisson',
 'Policier',
 'Politesse/Bienséance',
 'Politique',
 'Pollution',
 'Pologne',
 'Polynésie',
 'Pompier',
 'Port',
 'Portrait',
 'Portugal',
 'Poule/Poussin/Coq',
 'Poupée',
 'Pouvoir',
 'Poux',
 'Pratiques artistiques et culturelles',
 'Préhistoire',
 'Préjugés',
 'Prénom/Surnom',
 'Prince/Princesse',
 'Printemps',
 'Prison',
 'Prix littéraire',
 'Promenade',
 'Provence',
 'Proverbe',
 'Psychologie/Psychanalyse',
 'Publicité',
 'Puce',
 'Punition/Fessée',
 'Québec',
 'Questions',
 'Racisme',
 'Racket',
 'Réfugié',
 'Relation Enfant/Adulte',
 'Relation Enfant/Animal',
 'Relation Frère/Soeur',
 'Relation Grand-Mère/Enfant',
 'Relation Grand-Parent/Enfant',
 'Relation Grand-Père/Enfant',
 'Relation Homme/Animal',
 'Relation Mère/Enfant',
 'Relation Mère/Fille',
 'Relation Mère/Fils',
 'Relation Père/Enfant',
 'Relation Père/Fille',
 'Relation Père/Fils',
 'Relation Personne âgée/Enfant',
 'Religion',
 'Renard',
 'Résistance',
 'Respect',
 'Responsabilité',
 'Rêve',
 'Révolte',
 'Richesse',
 'Ritournelle/Répétition',
 'Rivalité/Dispute',
 'Robot',
 'Roi/Reine',
 'Roman',
 'Roman historique',
 'Roman policier',
 'Roumanie',
 'Rue',
 'Rumeur',
 'Ruse',
 'Russie',
 'Rwanda',
 'Sagesse',
 'Saint-Nicolas',
 'Saison/Année/Cycle des saisons',
 'Santé',
 'Sapin',
 'Science',
 'Science-Fiction',
 'Sculpture',
 'Secret',
 'Secte',
 'Sécurité routière',
 'Ségrégation raciale',
 'Sénégal',
 'Séparation/Divorce',
 'Serpent',
 'Sexualité/Education sexuelle',
 'Sida',
 'Silence',
 'Singe',
 'Société',
 'Soleil',
 'Solitude',
 'Sommeil',
 'Son/Eveil sonore',
 'Sorcier/Sorcellerie',
 'Souffrance',
 'Souris/Rat/Mulot',
 'Souvenir/Mémoire',
 'Spéléologie',
 'Spiritualité',
 'Sport',
 'Steampunk',
 'Sucre',
 'Suède',
 'Suicide',
 'Suisse',
 'Super-héros',
 'Superstition',
 'Surdité',
 'Survie',
 'Suspense',
 'Syrie',
 'Tabac/Tabagisme',
 'Taïwan',
 'Taupe',
 'Téléphone',
 'Télévision/Radio',
 'Temps',
 'Terre',
 'Terrorisme',
 'Thaïlande',
 'Théâtre',
 "Théâtre d'ombres",
 'Tibet',
 'Tigre',
 'Timidité',
 'Tolérance',
 'Tortue',
 'Toucher/Matière',
 'Tradition',
 'Traduction',
 'Train/Gare',
 'Transidentité',
 'Transport/Moyens de transport',
 'Travail',
 'Travail des enfants',
 'Trésor',
 'Trisomie',
 'Tristesse/Chagrin',
 'Tsigane',
 'Tunisie',
 'Turquie',
 'Typographie',
 'Tyrannie',
 'Ukraine',
 'Univers/Système solaire',
 'Vacances',
 'Vache',
 'Vampire',
 'Végétarisme/Végétalisme',
 'Vénézuela',
 'Vengeance',
 'Venise',
 'Vent',
 'Vêtement',
 'Vie quotidienne',
 'Vieillesse',
 'Vietnam',
 'Viking',
 'Ville',
 'Viol',
 'Violence',
 'Voisinage',
 'Vol/Voleur',
 'Volcan',
 'Voyage',
 'Vue',
 'Yéti',
 'Yoga',
 'Zèbre',
 'Zombie',
 'Zoo',
 'Zoologie/Naturalisme']

In [9]:
ricochet_pretraite = [word.lower() for word in ricochet]

In [10]:
ricochet_pretraite

["abandon/sentiment d'abandon",
 'abécédaire/alphabet',
 'abeille',
 'aborigène',
 'absence',
 'absurde',
 'abus sexuel',
 'accident',
 'activités manuelles',
 "activités/cahier d'activités",
 "adaptation (d'œuvre littéraire)",
 'adolescence',
 'adoption',
 'adultère',
 'affection/tendresse',
 'affirmation de soi',
 'afghanistan',
 'afrique',
 'afrique du nord',
 'afrique du sud',
 'aide humanitaire',
 'alaska',
 'album',
 'album sans texte',
 'alcool/alcoolisme',
 'algérie',
 'alimentation/goût',
 'allemagne',
 'alphabétisation',
 'alpinisme',
 'amérique du nord',
 'amérique latine',
 'amitié',
 'amnésie',
 'amour',
 'analyse de texte',
 'ane',
 'ange',
 'animaux',
 'anniversaire/fête',
 'anorexie/boulimie',
 'antilles/caraïbes',
 'antiquité (gaule)',
 'antiquité (grèce)',
 'antiquité (rome)',
 'antisémitisme',
 'apartheid',
 'apprentissage',
 'apprentissage de la vie',
 'araignée',
 'arbre',
 'archéologie',
 'architecture',
 'argent/argent de poche',
 'argentine',
 'arménie',
 'art',

In [11]:
len(ricochet)

716

## échantillonnage aléatoire

In [12]:
# éléments dans la v1 de l'ensemble de données (composé de 30 ricochets, 30 écoles des loisirs)
# listes utilisées pour que la v2 de l'ensemble de données ne contiennent pas la v1
ricochet_v1_exclure = ['ours', 'monde arabe', 'naissance/maternité', 'révolte', 'londres', 'papillon', 'souvenir/mémoire', 'eau', 'sénégal', 'dragon', 'hygiène/propreté', "atelier d'écriture", 'alaska', 'poisson', 'astronomie', 'documentaire', 'débarquement', 'guatemala', 'sida', "lecture de l'image", 'pompier', 'nuit', 'univers/système solaire', 'peintre', 'souffrance', 'aide humanitaire', 'maltraitance', 'finlande', 'licorne', 'edition']
ecole_des_loisirs_v1_exclure = ['conte détourné', 'histoire : 18e siècle', 'aventure et mystère', 'théâtre contemporain', 'tchécoslovaquie', 'oman', 'genre', 'recherche de ses origines', 'préhistoire', 'père noël', 'rugby', 'balle, ballon', 'bolivie', 'roller', 'animaux domestiques', 'yémen', 'écrivain, écriture', 'biographie', 'hippocampe', 'arts martiaux', 'semaine', 'livre-jeu', 'histoire sans texte', 'cow-boy', 'transhumanisme', "conte détourné, boucle d'or et les 3 ours", 'relation enfant, adulte', 'conte détourné, le loup et les 7 cabris', 'regard des autres', 'iran']

In [13]:
# Echantillon aléatoire de ricochet_raw en omettant les éléments de omit_list
ricochet_samples = random.sample([word for word in ricochet_pretraite if word not in ricochet_v1_exclure], 20)

print("Echantillon aléatoire de ricochet_raw en omettant les éléments de omit_list")
print(len(ricochet_samples), ricochet_samples)

# Echantillon aléatoire de ecole_des_loisirs_raw en omettant les éléments de omit_list
ecole_des_loisirs_samples = random.sample([word for word in ecole_des_loisirs if word not in ecole_des_loisirs_v1_exclure], 20)

print("Echantillon aléatoire de ecole_des_loisirs_raw en omettant les éléments de omit_list")
print(len(ecole_des_loisirs_samples), ecole_des_loisirs_samples)

Echantillon aléatoire de ricochet_raw en omettant les éléments de omit_list
20 ['exclusion', 'fantaisie', 'fable', 'droit', 'irak', 'catastrophe naturelle', 'clown', 'enlèvement/kidnapping', 'pêche/pêcheur', 'magasin/commerce', 'ecosse', 'belgique', 'transport/moyens de transport', "abandon/sentiment d'abandon", 'colonisation/colonie', 'train/gare', 'travail', 'imagier', 'printemps', 'roman historique']
Echantillon aléatoire de ecole_des_loisirs_raw en omettant les éléments de omit_list
20 ['ange', 'Littérature, livre, écrivain', 'feu', 'relation avec la mère', 'des animaux et des hommes', 'cervidés', 'camion, voiture', 'insectes divers', 'conquête', 'Noël', 'conditions sociales', 'famille monoparentale', 'bêtises, désobéissance', 'responsable', 'méchanceté, cruauté', 'gratitude', 'musée', 'camping', 'rivière, fleuve, étang', "protection de l'environnement"]


## Assignation des évaluateurs
2 évaluateurs par échantillon pour 3 évaluateurs

In [14]:
def assigner_evaluateurs(echantillons):
    evaluateurs = ['Juliette', 'Michel', 'Karim']

    # Create a cycle iterator for the reviewers
    cycles_evaluateurs = itertools.cycle(evaluateurs)

    # Assign reviewers to each sample with a UUID
    assignations = {}
    for sample in echantillons:
        evaluateurs = [next(cycles_evaluateurs) for _ in range(2)] # Assign 2 reviewers per sample
        id_assignation = uuid.uuid4()
        assignations[id_assignation] = {'sample': sample, 'reviewers': evaluateurs}

    return assignations

# Example usage:
assignations_ricochet = assigner_evaluateurs(ricochet_samples)
assignations_ecole_des_loisirs = assigner_evaluateurs(ecole_des_loisirs_samples)


In [15]:
assignations_ricochet

{UUID('3f4f22f8-f220-494a-bed4-2d90e29c8a80'): {'sample': 'exclusion',
  'reviewers': ['Juliette', 'Michel']},
 UUID('a9ef994d-2936-4609-91e9-713fdf0cc9c0'): {'sample': 'fantaisie',
  'reviewers': ['Karim', 'Juliette']},
 UUID('954096a6-085e-4166-9344-57b0c16dcd56'): {'sample': 'fable',
  'reviewers': ['Michel', 'Karim']},
 UUID('026321bc-6781-4026-bfd8-8001641c287b'): {'sample': 'droit',
  'reviewers': ['Juliette', 'Michel']},
 UUID('4eed0e6f-3697-45df-acca-770d30e3eb18'): {'sample': 'irak',
  'reviewers': ['Karim', 'Juliette']},
 UUID('2d4f83e6-eb1d-40bd-8926-7d27eadf4e09'): {'sample': 'catastrophe naturelle',
  'reviewers': ['Michel', 'Karim']},
 UUID('d170f252-2a47-4403-a8de-842f5153786d'): {'sample': 'clown',
  'reviewers': ['Juliette', 'Michel']},
 UUID('4436c43b-1f02-420b-a4b2-75e379d37e7b'): {'sample': 'enlèvement/kidnapping',
  'reviewers': ['Karim', 'Juliette']},
 UUID('a3af426a-15f7-4774-bce9-461bd1528286'): {'sample': 'pêche/pêcheur',
  'reviewers': ['Michel', 'Karim']},
 U

In [16]:
assignations_ecole_des_loisirs

{UUID('4654ac02-49d4-4e8f-82a9-d7fb1bf445a5'): {'sample': 'ange',
  'reviewers': ['Juliette', 'Michel']},
 UUID('564d9c20-8bc5-473b-913a-668c37d87960'): {'sample': 'Littérature, livre, écrivain',
  'reviewers': ['Karim', 'Juliette']},
 UUID('5b554435-8c06-4791-9100-6c8f066bfab5'): {'sample': 'feu',
  'reviewers': ['Michel', 'Karim']},
 UUID('5ed90b2f-0b50-422e-8718-dbf7312302c1'): {'sample': 'relation avec la mère',
  'reviewers': ['Juliette', 'Michel']},
 UUID('6589d91b-19c2-45a7-ae72-67709d891577'): {'sample': 'des animaux et des hommes',
  'reviewers': ['Karim', 'Juliette']},
 UUID('33b0392a-5e06-48a0-9954-72d2912ff109'): {'sample': 'cervidés',
  'reviewers': ['Michel', 'Karim']},
 UUID('fd8d90f7-b53a-4af8-ac83-6ab1cbb8da39'): {'sample': 'camion, voiture',
  'reviewers': ['Juliette', 'Michel']},
 UUID('a5b2eada-2278-4686-86a9-4e1c279d9944'): {'sample': 'insectes divers',
  'reviewers': ['Karim', 'Juliette']},
 UUID('481fd867-bf16-4f91-a1d5-c79282c91985'): {'sample': 'conquête',
  'r

## brassage aléatoire des assignations

In [17]:
assignations_brassees = list(assignations_ricochet.items()) + list(assignations_ecole_des_loisirs.items())
random.shuffle(assignations_brassees)

In [18]:
assignations_brassees

[(UUID('5b554435-8c06-4791-9100-6c8f066bfab5'),
  {'sample': 'feu', 'reviewers': ['Michel', 'Karim']}),
 (UUID('c0f9a4b0-7671-434d-a116-452602feed1a'),
  {'sample': 'responsable', 'reviewers': ['Karim', 'Juliette']}),
 (UUID('7eaf3dc1-1678-4315-89ea-0b1c5eb2b54b'),
  {'sample': 'bêtises, désobéissance', 'reviewers': ['Juliette', 'Michel']}),
 (UUID('20d47e46-5564-4534-8ee5-78a4b943c68a'),
  {'sample': 'colonisation/colonie', 'reviewers': ['Michel', 'Karim']}),
 (UUID('a9ef994d-2936-4609-91e9-713fdf0cc9c0'),
  {'sample': 'fantaisie', 'reviewers': ['Karim', 'Juliette']}),
 (UUID('4eed0e6f-3697-45df-acca-770d30e3eb18'),
  {'sample': 'irak', 'reviewers': ['Karim', 'Juliette']}),
 (UUID('6589d91b-19c2-45a7-ae72-67709d891577'),
  {'sample': 'des animaux et des hommes', 'reviewers': ['Karim', 'Juliette']}),
 (UUID('ff269e93-f214-4d2a-88a1-bf4e83de001e'),
  {'sample': 'ecosse', 'reviewers': ['Karim', 'Juliette']}),
 (UUID('dc6d5028-a5e7-4df9-af96-2604bf533078'),
  {'sample': "protection de l'e

## generation du fichier excel pour les evaluateurs

In [19]:
def ecrire_donnees_dans_excel(assignations, evaluateur):
    donnees_a_ecrire = []
    for id_assignations, info_assignation in assignations:
        sample = info_assignation['sample']
        evaluateurs = info_assignation['reviewers']
        if evaluateur in evaluateurs:
            donnees_a_ecrire.append({'UUID': id_assignations,
                        'Echantillon': sample, 
                        'Resultat 1': '', 
                        'Resultat 2': ''})
    
    df = pd.DataFrame(donnees_a_ecrire, columns=['UUID', 'Echantillon', 'Resultat 1', 'Resultat 2'])
    df.to_excel(f"{evaluateur}_assignations_v2.xlsx", index=False)   
    print("donneés ecrites")     


# decommenter pour ecrire
# ecrire_donnees_dans_excel(assignations_brassees, 'Juliette')
# ecrire_donnees_dans_excel(assignations_brassees, 'Michel')
# ecrire_donnees_dans_excel(assignations_brassees, 'Karim')


## excel qui réunit les assignations de tous les evaluateurs 

In [20]:
filename = 'assignations_generales_v2.xlsx' # fonction appelée 2 fois pour peupler le meme fichier
if os.path.exists(filename):
    os.remove(filename)

def peupler_donnees(donnees, assignations, source):
    for id_assignation, info_assignation in assignations.items():
        echantillon = info_assignation['sample']
        evaluateur = info_assignation['reviewers']
        donnees.append({'Uuid': id_assignation,
                     'Source': source, 
                     'Echantillon': echantillon, 
                     'Evaluateur 1': evaluateur[0], 
                     "Evaluateur 2": evaluateur[1], 
                     'Resultat 1': '', 
                     'Resultat 2': ''})
    return donnees
    
def ecrire_donnees_dans_excel_principal():
    # Define the data as a list of dictionaries

    donnees = []   
    donnees = peupler_donnees(donnees, assignations_ricochet, 'ricochet')
    donnees = peupler_donnees(donnees, assignations_ecole_des_loisirs, 'ecoles des loisirs')

    # Write the data to a CSV file
    data_frame = pd.DataFrame(donnees)
    data_frame.to_excel(filename, index=False)
    print("donneés ecrites")     

# decommenter pour ecrire
# ecrire_donnees_dans_excel_principal()


# lecture des jeux de données annotées sur excel

In [21]:
# Définir les options d'affichage pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

In [31]:
# df = dataframe = objet pandas qui contient les données
df_michel = pd.read_excel('../book_themes_alignment_data/Michel_assignments_v1_v2.xlsx', keep_default_na=False, na_values=[''])
df_michel.rename(columns={'Echantillon': 'Echantillon (Michel)', 'Resultat 1': 'Resultat 1 (Michel)', 'Resultat 2': 'Resultat 2 (Michel)'}, inplace=True)
# print(df_michel)

In [32]:
df_karim = pd.read_excel('../book_themes_alignment_data/Karim_assignments_v1_v2.xlsx', keep_default_na=False, na_values=[''])
df_karim.rename(columns={'Echantillon': 'Echantillon (Karim)','Resultat 1': 'Resultat 1 (Karim)', 'Resultat 2': 'Resultat 2 (Karim)'}, inplace=True)
# print(df_karim)

In [33]:
df_juliette = pd.read_excel('../book_themes_alignment_data/Juliette_assignments_v1_v2.xlsx', keep_default_na=False, na_values=[''])
df_juliette.rename(columns={'Echantillon': 'Echantillon (Juliette)', 'Resultat 1': 'Resultat 1 (Juliette)', 'Resultat 2': 'Resultat 2 (Juliette)'}, inplace=True)
# print(df_juliette)

In [34]:
# Fusionner les dataframes
df_fusion = pd.concat([df_michel.set_index('UUID'), df_juliette.set_index('UUID'), df_karim.set_index('UUID')], axis=1)

echantillons_fusionnés = pd.concat([df_fusion["Echantillon (Juliette)"], df_fusion["Echantillon (Karim)"], df_fusion["Echantillon (Michel)"]]).dropna()
echantillons_fusionnés = echantillons_fusionnés.drop_duplicates()
df_fusion['Echantillon'] = echantillons_fusionnés
df_fusion.drop(columns=['Echantillon (Juliette)', 'Echantillon (Karim)', 'Echantillon (Michel)'], inplace=True)

In [35]:
df_michel_karim = df_fusion.loc[:, ['Echantillon', 'Resultat 1 (Michel)', 'Resultat 1 (Karim)']]
df_michel_karim = df_michel_karim.dropna()
print(df_michel_karim)

                                                  Echantillon Resultat 1 (Michel) Resultat 1 (Karim)
UUID                                                                                                
0f660312-e967-40fe-9a52-2cbed0ca23f8            balle, ballon                  SF                 SF
1885575b-e89a-4d0b-8474-9b98b049e634      animaux domestiques                 WNG                WNG
0d8703ec-db50-4936-9808-3d1e3befafe3                  sénégal               1HFDS              1HFDS
fbad8376-7a5b-4e0a-95d3-8edfda117779       atelier d'écriture                 CBW                CBW
3c436173-9bb3-47f5-80ab-405328bb397f                  edition               KNTP1              KNTP1
0703fbf6-f436-4cf6-913a-49295b511ccf             maltraitance                YXQD               YXQD
564404fe-7d19-447e-b4c7-b257188ee07b               biographie                 DNB                YNB
6e702f37-fd76-45a1-8ae1-3b9be5bb5bc4      aventure et mystère                YFCF          

In [36]:
df_michel_juliette = df_fusion.loc[:, ['Echantillon', 'Resultat 1 (Michel)', 'Resultat 1 (Juliette)']]
df_michel_juliette = df_michel_juliette.dropna()
print(df_michel_juliette)

                                                                  Echantillon Resultat 1 (Michel) Resultat 1 (Juliette)
UUID                                                                                                                   
a2492fac-993c-46c2-817b-67bf4b9fd16f                                livre-jeu               YBCS2                   YBG
f717074b-f95b-4ac7-a704-16ec89e8850b                                     ours                WNCF               YNNJV23
83d95dc7-2a1d-45e6-a6b0-efdabb296555                     théâtre contemporain                 DDC                   DDC
9dd6ba61-b47c-443a-8cab-7dc358fd1eeb                           transhumanisme                 UBJ                   FLP
be6e6fe5-9ea4-410a-ab44-b4e78e8339c3                                    genre                JBSF                   YXC
71f9881c-fbc5-4c5e-9c1d-983d93f35315                                   alaska        1KBB-US-WPNA          1KBB-US-WPNA
74977d39-4cd9-4abc-b91d-44150fe57676    

In [37]:
df_karim_juliette = df_fusion.loc[:, ['Echantillon', 'Resultat 1 (Karim)', 'Resultat 1 (Juliette)']]
df_karim_juliette = df_karim_juliette.dropna()
print(df_karim_juliette)

                                                                    Echantillon Resultat 1 (Karim) Resultat 1 (Juliette)
UUID                                                                                                                    
6122a480-817b-445a-b993-65e65accaa50                    univers/système solaire                PGM                   PGS
e9aabaf3-9be3-42cc-a210-e955048a7468                         écrivain, écriture               DNBL                   YNL
56a52e12-2040-4303-af7e-e921cd6dc7f5                      histoire : 18e siècle                YNH                    NH
1d4bf095-0667-4683-80e4-b9981e1b967e                               débarquement               JWCK                  JWCK
eb85b54c-76cd-4a63-84a2-dffa891bebd4                           aide humanitaire               JKSR                  JKSR
6383a51e-fcce-496d-9b4f-008a44c7570c                                      rugby              YNWD7                 YNWD7
efe3f8c2-80fe-4f90-ba11-8b048b70

# kappa: comparaison des annotation entre les différents évaluateurs

In [38]:
michel = df_michel_karim["Resultat 1 (Michel)"].tolist()
karim = df_michel_karim["Resultat 1 (Karim)"].tolist()

kappa = sklearn.metrics.cohen_kappa_score(michel, karim)
print("Michel & Karim", kappa, len(michel))

Michel & Karim 0.4756871035940803 31


In [39]:
michel = df_michel_juliette["Resultat 1 (Michel)"].tolist()
juliette = df_michel_juliette["Resultat 1 (Juliette)"].tolist()

kappa = sklearn.metrics.cohen_kappa_score(michel, juliette)
print("Michel & Juliette", kappa, len(michel))

Michel & Juliette 0.5211267605633803 34


In [43]:
karim = df_karim_juliette["Resultat 1 (Karim)"].tolist()
juliette = df_karim_juliette["Resultat 1 (Juliette)"].tolist()

kappa = sklearn.metrics.cohen_kappa_score(karim, juliette)
print("Juliette & Karim", kappa, len(karim))

Juliette & Karim 0.44641192917054984 33


# creation d'un dictionnaire pour les données de références (DR)
[clé=nom_theme, valeur=list(code_theme)]

In [49]:
theme_dr = {} # dr = données référence
for index, row in df_fusion.iterrows():
    resultats_ligne_df = row.filter(like='Resultat').dropna()
    theme_dr[row["Echantillon"]] = resultats_ligne_df.tolist()
theme_dr

{'balle, ballon': ['SF', 'SF'],
 'animaux domestiques': ['WNG', 'WNG'],
 'sénégal': ['1HFDS', '1HFDS'],
 'livre-jeu': ['YBCS2', 'YBG'],
 "atelier d'écriture": ['CBW', 'CBW'],
 'edition': ['KNTP1', 'KNTP1'],
 'ours': ['WNCF', 'YNNJV23'],
 'théâtre contemporain': ['DDC', 'DDC'],
 'maltraitance': ['YXQD', 'YXQD'],
 'transhumanisme': ['UBJ', 'FLP', 'UBJ'],
 'genre': ['JBSF', 'YXC'],
 'alaska': ['1KBB-US-WPNA', '1KBB-US-WPNA'],
 'biographie': ['DNB', 'YNB'],
 'bolivie': ['1KLSL', '1KLSL'],
 'conte détourné, le loup et les 7 cabris': ['YFHG', 'YFHG'],
 'aventure et mystère': ['YFCF', 'YFC', 'YNX'],
 'conte détourné': ['YFHG', 'YFHG'],
 'yémen': ['1FBXY', '1FBXY'],
 'hippocampe': ['WNCS1', 'N/A'],
 'cow-boy': ['FJW', 'FRJ'],
 'dragon': ['VXQM1', 'YNXB1'],
 'père noël': ['5HPD', 'N/A'],
 'sida': ['MJCJ2', 'MJCJ2'],
 'souvenir/mémoire': ['JMR', 'VSPT'],
 'pompier': ['JKSW2', 'JKSW2'],
 'iran': ['1FBN', '1FBN'],
 'semaine': ['YBLJ', '5HCK'],
 'finlande': ['1DNF', '1DNF'],
 'peintre': ['AGB', 'YN

In [54]:
import json

chemin_fichier = '../book_themes_alignment_data/themes_jeu_validation_donnees_references.json'

# Save the JSON string to a file
with open(chemin_fichier, 'w') as file:
    json.dump(theme_dr, file)

# Print the file path
print(f"Données références sauvegardées à: {chemin_fichier}")


Données références sauvegardées à: ../book_themes_alignment_data/themes_jeu_validation_donnees_references.json


# alignement thesaurus

## import données références

In [56]:
chemin_fichier = '../book_themes_alignment_data/themes_jeu_validation_donnees_references.json'

with open(chemin_fichier, "r") as file:
    themes_jeu_validation_dr = json.load(file) # dr = données references

themes_jeu_validation_dr

{'balle, ballon': ['SF', 'SF'],
 'animaux domestiques': ['WNG', 'WNG'],
 'sénégal': ['1HFDS', '1HFDS'],
 'livre-jeu': ['YBCS2', 'YBG'],
 "atelier d'écriture": ['CBW', 'CBW'],
 'edition': ['KNTP1', 'KNTP1'],
 'ours': ['WNCF', 'YNNJV23'],
 'théâtre contemporain': ['DDC', 'DDC'],
 'maltraitance': ['YXQD', 'YXQD'],
 'transhumanisme': ['UBJ', 'FLP', 'UBJ'],
 'genre': ['JBSF', 'YXC'],
 'alaska': ['1KBB-US-WPNA', '1KBB-US-WPNA'],
 'biographie': ['DNB', 'YNB'],
 'bolivie': ['1KLSL', '1KLSL'],
 'conte détourné, le loup et les 7 cabris': ['YFHG', 'YFHG'],
 'aventure et mystère': ['YFCF', 'YFC', 'YNX'],
 'conte détourné': ['YFHG', 'YFHG'],
 'yémen': ['1FBXY', '1FBXY'],
 'hippocampe': ['WNCS1', 'N/A'],
 'cow-boy': ['FJW', 'FRJ'],
 'dragon': ['VXQM1', 'YNXB1'],
 'père noël': ['5HPD', 'N/A'],
 'sida': ['MJCJ2', 'MJCJ2'],
 'souvenir/mémoire': ['JMR', 'VSPT'],
 'pompier': ['JKSW2', 'JKSW2'],
 'iran': ['1FBN', '1FBN'],
 'semaine': ['YBLJ', '5HCK'],
 'finlande': ['1DNF', '1DNF'],
 'peintre': ['AGB', 'YN

## import des noeuds thema

## prétraitement : prendre le premier mot, le réduire et supprimer les caractères spéciaux

In [66]:
with open("../book_themes_alignment_data/thema_code_dict.json", "r") as file:
    thema_code_dict = json.load(file)

thema_code_dict

{'A': {'CodeValue': 'A',
  'CodeDescription': 'Arts',
  'CodeNotes': 'Utilisez tous les codes A* pour les ouvrages spécialisés et généraux, qu’ils soient richement illustrés ou majoritairement textuel. Préférez les codes WF* pour les ouvrages liés à un loisir ou un passe-temps, en le complétant au besoin par le(s) code(s) A*. Utilisez au besoin tous les codes A* avec d’autres codes et qualificateurs, particulièrement avec les qualificateurs de style 6*, de lieux 1* et historiques 3*',
  'CodeParent': '',
  'IssueNumber': 1,
  'Modified': 1.4},
 'AB': {'CodeValue': 'AB',
  'CodeDescription': 'Arts : généralités',
  'CodeNotes': '',
  'CodeParent': 'A',
  'IssueNumber': 1,
  'Modified': 1.5},
 'ABA': {'CodeValue': 'ABA',
  'CodeDescription': 'Théorie de l’art',
  'CodeNotes': 'Lié : QDTN',
  'CodeParent': 'AB',
  'IssueNumber': 1,
  'Modified': ''},
 'ABC': {'CodeValue': 'ABC',
  'CodeDescription': 'Conservation, restauration et entretien d’œuvres d’art',
  'CodeNotes': 'Utilisez tous le

In [95]:
noeuds_thema_pretraites = {}
for code, theme in thema_code_dict.items():
    # Preprocess the code description
    noeuds_thema_pretraites[code] = copy.deepcopy(theme)
    noeuds_thema_pretraites[code]["CodeDescription"] = theme["CodeDescription"].lower()
    noeuds_thema_pretraites[code]["CodeNotes"] = theme["CodeNotes"].lower()

noeuds_thema_pretraites

{'A': {'CodeValue': 'A',
  'CodeDescription': 'arts',
  'CodeNotes': 'utilisez tous les codes a* pour les ouvrages spécialisés et généraux, qu’ils soient richement illustrés ou majoritairement textuel. préférez les codes wf* pour les ouvrages liés à un loisir ou un passe-temps, en le complétant au besoin par le(s) code(s) a*. utilisez au besoin tous les codes a* avec d’autres codes et qualificateurs, particulièrement avec les qualificateurs de style 6*, de lieux 1* et historiques 3*',
  'CodeParent': '',
  'IssueNumber': 1,
  'Modified': 1.4},
 'AB': {'CodeValue': 'AB',
  'CodeDescription': 'arts : généralités',
  'CodeNotes': '',
  'CodeParent': 'A',
  'IssueNumber': 1,
  'Modified': 1.5},
 'ABA': {'CodeValue': 'ABA',
  'CodeDescription': 'théorie de l’art',
  'CodeNotes': 'lié : qdtn',
  'CodeParent': 'AB',
  'IssueNumber': 1,
  'Modified': ''},
 'ABC': {'CodeValue': 'ABC',
  'CodeDescription': 'conservation, restauration et entretien d’œuvres d’art',
  'CodeNotes': 'utilisez tous le

## alignement thesaurus naif: correspondance exacte

In [90]:
def evaluation_correspondance_exacte(thema_dict):
    evaluations_positives_correspondance_exacte = {}
    compte_correspondances = 0
    correspondances = []
    jeu_donnees_complet = []

    for theme_a_evaluer in themes_jeu_validation_dr.keys():
        jeu_donnees_complet.append(theme_a_evaluer)
        for thema_code, thema_valeur in thema_dict.items():
            thema_valeur = thema_valeur["CodeDescription"]
            if theme_a_evaluer == thema_valeur and theme_a_evaluer not in evaluations_positives_correspondance_exacte.values():
                correspondances.append(theme_a_evaluer)
                print(theme_a_evaluer, "->", thema_valeur)
                compte_correspondances += 1
                evaluations_positives_correspondance_exacte[theme_a_evaluer] = thema_code
    
    sans_correspondances = list(set(jeu_donnees_complet) - set(correspondances))
    print("jeu données complet", len(jeu_donnees_complet))
    print(jeu_donnees_complet)
    print("correspondances", len(correspondances))
    print(correspondances)
    print("sans correspondance", len(sans_correspondances))
    print(sans_correspondances)

    return evaluations_positives_correspondance_exacte
evaluations_positives_correspondance_exacte = evaluation_correspondance_exacte(noeuds_thema_pretraites)

sénégal -> sénégal
alaska -> alaska
bolivie -> bolivie
yémen -> yémen
iran -> iran
finlande -> finlande
guatemala -> guatemala
préhistoire -> préhistoire
oman -> oman
arts martiaux -> arts martiaux
tchécoslovaquie -> tchécoslovaquie
jeu données complet 60
['balle, ballon', 'animaux domestiques', 'sénégal', 'livre-jeu', "atelier d'écriture", 'edition', 'ours', 'théâtre contemporain', 'maltraitance', 'transhumanisme', 'genre', 'alaska', 'biographie', 'bolivie', 'conte détourné, le loup et les 7 cabris', 'aventure et mystère', 'conte détourné', 'yémen', 'hippocampe', 'cow-boy', 'dragon', 'père noël', 'sida', 'souvenir/mémoire', 'pompier', 'iran', 'semaine', 'finlande', 'peintre', 'relation enfant, adulte', 'papillon', 'souffrance', 'nuit', 'guatemala', 'préhistoire', 'astronomie', 'révolte', 'naissance/maternité', 'documentaire', 'oman', 'univers/système solaire', 'écrivain, écriture', 'histoire : 18e siècle', 'débarquement', 'aide humanitaire', 'rugby', 'histoire sans texte', 'poisson', 

In [106]:
import sys
from pprint import pprint
epsilon = sys.float_info.epsilon


def calculer_metriques(evaluations_positives):
    print("len(theme evaluation)", len(evaluations_positives))  # contient les evaluations positives du modele
    print("len(themes_jeu_validation_dr)", len(themes_jeu_validation_dr)) # contient les données de références de tout le jeu de données de validation

    VP = epsilon # eviter division par 0
    VP_liste = []
    FP = 0
    FP_liste = []
    for evaluation_theme_nom, evaluation_theme_code in evaluations_positives.items():
        if evaluation_theme_nom in themes_jeu_validation_dr and evaluation_theme_code in themes_jeu_validation_dr[evaluation_theme_nom]:
            VP_liste.append((evaluation_theme_nom, evaluation_theme_code, themes_jeu_validation_dr[evaluation_theme_nom]))
            VP += 1
        elif evaluation_theme_nom in themes_jeu_validation_dr and evaluation_theme_code not in themes_jeu_validation_dr[evaluation_theme_nom]:
            evaluation_theme_name_in_thema = thema_code_dict[evaluation_theme_code]["CodeDescription"] # name of the evaluated theme in thema
            validation_set_key = themes_jeu_validation_dr[evaluation_theme_nom][0]
            if validation_set_key != 'N/A' :
                validation_set_name = thema_code_dict[validation_set_key]["CodeDescription"] # name of the ground truth theme in thema
                FP_liste.append(f"{evaluation_theme_nom} -> {evaluation_theme_name_in_thema} vs φ={validation_set_name} | {evaluation_theme_code} vs φ={validation_set_key}")
            else:
                FP_liste.append(f"{evaluation_theme_nom} -> {evaluation_theme_name_in_thema} vs φ=N/A | {evaluation_theme_code} vs φ={validation_set_key}")


            FP += 1

            
    assert(FP == len(evaluations_positives) - VP) # FP = P - VP
    FN = len(themes_jeu_validation_dr) - len(evaluations_positives)  # FN = N =  tout le jeu de donneés - P
    assert(len(themes_jeu_validation_dr) == VP + FN + FP)
    print("VP", VP)
    pprint(VP_liste)
    print("FP", FP)
    pprint(FP_liste)
    print("FN", FN)


    accuracy = (VP / len(themes_jeu_validation_dr))
    precision = VP / (VP + FP)
    recall = VP / (VP + FN)
    f1_score = 2 * (precision * recall) / (precision + recall)

    print("precision", precision)
    print("rappel", recall)
    print("f1_score", f1_score)
    print("accuracy", accuracy)


calculer_metriques(evaluations_positives_correspondance_exacte)

len(theme evaluation) 11
len(themes_jeu_validation_dr) 60
VP 10.0
[('sénégal', '1HFDS', ['1HFDS', '1HFDS']),
 ('alaska', '1KBB-US-WPNA', ['1KBB-US-WPNA', '1KBB-US-WPNA']),
 ('bolivie', '1KLSL', ['1KLSL', '1KLSL']),
 ('yémen', '1FBXY', ['1FBXY', '1FBXY']),
 ('iran', '1FBN', ['1FBN', '1FBN']),
 ('finlande', '1DNF', ['1DNF', '1DNF']),
 ('guatemala', '1KLCG', ['1KLCG', '1KLCG']),
 ('préhistoire', '3B', ['3B', '3B']),
 ('oman', '1FBXM', ['1FBXM', '1FBXM']),
 ('tchécoslovaquie', '1QBDK', ['1QBDK', '1QBDK'])]
FP 1
['arts martiaux -> arts martiaux vs φ=documentaires jeunesse : arts martiaux | '
 'SRM vs φ=YNWJ']
FN 49
precision 0.9090909090909091
rappel 0.1694915254237288
f1_score 0.2857142857142857
accuracy 0.16666666666666666


## levenshtein

In [96]:
import Levenshtein
def levenstein_evaluation(thema_dict):
    theme_evaluation_levenstein = {}
    match_count = 0
    matches = []
    all_data = []

    for theme_to_evaluate in themes_jeu_validation_dr.keys():
        # UPPER_DISTANCE_LIMIT = max(len(theme_to_evaluate) / 3, 0)
        UPPER_DISTANCE_LIMIT = 1
        all_data.append(theme_to_evaluate)
        min_distance = 100
        best_match = None
        for thema_code, thema_value in thema_dict.items():
            thema_value = thema_value["CodeDescription"]
            distance = Levenshtein.distance(theme_to_evaluate, thema_value)
            if distance < min_distance and distance <= UPPER_DISTANCE_LIMIT:
                min_distance = distance
                best_match = thema_code
        if min_distance <= UPPER_DISTANCE_LIMIT and best_match is not None and theme_to_evaluate not in theme_evaluation_levenstein.values():
            matches.append(theme_to_evaluate)
            print(min_distance, theme_to_evaluate, "->", thema_dict[best_match]["CodeDescription"])
            match_count += 1
            theme_evaluation_levenstein[theme_to_evaluate] = best_match
    
    non_matches = list(set(all_data) - set(matches))
    print("all_data", len(all_data))
    print(all_data)
    print("matches", len(matches))
    print(matches)
    print("non_matches", len(non_matches))
    print(non_matches)

    return theme_evaluation_levenstein

theme_evaluation_levenstein = levenstein_evaluation(noeuds_thema_pretraites)


0 sénégal -> sénégal
1 ours -> tours
0 alaska -> alaska
0 bolivie -> bolivie
0 yémen -> yémen
0 iran -> iran
0 finlande -> finlande
0 guatemala -> guatemala
0 préhistoire -> préhistoire
1 documentaire -> documentaires
0 oman -> oman
0 arts martiaux -> arts martiaux
0 tchécoslovaquie -> tchécoslovaquie
1 eau -> pau
all_data 60
['balle, ballon', 'animaux domestiques', 'sénégal', 'livre-jeu', "atelier d'écriture", 'edition', 'ours', 'théâtre contemporain', 'maltraitance', 'transhumanisme', 'genre', 'alaska', 'biographie', 'bolivie', 'conte détourné, le loup et les 7 cabris', 'aventure et mystère', 'conte détourné', 'yémen', 'hippocampe', 'cow-boy', 'dragon', 'père noël', 'sida', 'souvenir/mémoire', 'pompier', 'iran', 'semaine', 'finlande', 'peintre', 'relation enfant, adulte', 'papillon', 'souffrance', 'nuit', 'guatemala', 'préhistoire', 'astronomie', 'révolte', 'naissance/maternité', 'documentaire', 'oman', 'univers/système solaire', 'écrivain, écriture', 'histoire : 18e siècle', 'débarq

In [97]:
calculer_metriques(theme_evaluation_levenstein)

len(theme evaluation) 14
len(themes_jeu_validation_dr) 60
VP 11.0
[('sénégal', '1HFDS', ['1HFDS', '1HFDS']),
 ('alaska', '1KBB-US-WPNA', ['1KBB-US-WPNA', '1KBB-US-WPNA']),
 ('bolivie', '1KLSL', ['1KLSL', '1KLSL']),
 ('yémen', '1FBXY', ['1FBXY', '1FBXY']),
 ('iran', '1FBN', ['1FBN', '1FBN']),
 ('finlande', '1DNF', ['1DNF', '1DNF']),
 ('guatemala', '1KLCG', ['1KLCG', '1KLCG']),
 ('préhistoire', '3B', ['3B', '3B']),
 ('documentaire', 'ATFR', ['YN', 'ATFR']),
 ('oman', '1FBXM', ['1FBXM', '1FBXM']),
 ('tchécoslovaquie', '1QBDK', ['1QBDK', '1QBDK'])]
FP 3
['ours -> tours vs φ=vie sauvage : mammifères : généralités | 1DDF-FR-GDA vs '
 'φ=WNCF',
 'arts martiaux -> arts martiaux vs φ=documentaires jeunesse : arts martiaux | '
 'SRM vs φ=YNWJ',
 'eau -> pau vs φ=hydrologie et hydrosphère | 1DDF-FR-BEA vs φ=RBK']
FN 46
precision 0.7857142857142857
rappel 0.19298245614035087
f1_score 0.30985915492957744
accuracy 0.18333333333333332


## wiki2vec

In [98]:
from wikipedia2vec import Wikipedia2Vec
# download from https://wikipedia2vec.github.io/wikipedia2vec/pretrained/
# Load the pre-trained model
wiki2vec = Wikipedia2Vec.load('../book_themes_alignment_data/frwiki_20180420_300d.pkl')

# Find the most similar words to a given word
similar_words = wiki2vec.most_similar(wiki2vec.get_word('chien'), 10)

# Print the similar words
for word, similarite in similar_words:
    print(f"{word}: {similarite}")

c:\Users\dre\Documents\projet biblio scolaire\Books-Python\books-python-env\Lib\site-packages\joblib\numpy_pickle.py:230: UserWarning: The memmapped array [[-0.3918146  -0.24315855 -0.29823107 ... -0.15211841  0.34864503
  -0.28443316]
 [-0.31431052 -0.05210092 -0.23165612 ...  0.33593094  0.21336684
  -0.2881443 ]
 [ 0.23390982 -0.00242238 -0.24155504 ...  0.10083728 -0.06711676
  -0.03356545]
 ...
 [-0.37162766 -0.5298218  -0.05777219 ... -0.2812191   0.15711616
   0.2854844 ]
 [-0.75576866 -0.14604965 -0.24931757 ... -0.32277733 -0.1752458
   0.3595142 ]
 [-0.4601417  -0.19192158 -0.281166   ... -0.36144966 -0.00496797
   0.08007599]] loaded from the file ../book_themes_alignment_data/frwiki_20180420_300d.pkl is not byte aligned. This may cause segmentation faults if this memmapped array is used in some libraries like BLAS or PyTorch. To get rid of this warning, regenerate your pickle file with joblib >= 1.2.0. See https://github.com/joblib/joblib/issues/563 for more details
  warni

<Word chien>: 1.0000001192092896
<Word chiens>: 0.7172577381134033
<Word chiot>: 0.6617892980575562
<Word chat>: 0.658769428730011
<Word pudelpointer>: 0.6494312286376953
<Word chienne>: 0.6439096927642822
<Entity Chien de chasse>: 0.6328124403953552
<Word caniche>: 0.6294683814048767
<Word drahthaar>: 0.6275166869163513
<Word snuppy>: 0.6268122792243958


In [100]:
def align_lists_with_synonyms_wiki2vec(thema_dict, wiki2vec):

    theme_evaluation_wiki2vec = {}
    MINIMUM_COSINE_SIMILARITY = -1

    for theme_to_evaluate in themes_jeu_validation_dr.keys():
        # Calculate the similarity between each pair of words
        max_similarity = MINIMUM_COSINE_SIMILARITY
        best_match = None
        for thema_code, thema_value in thema_dict.items():
            thema_value = thema_value["CodeDescription"] # name of thema theme
            try:
                word1 = wiki2vec.get_word_vector(theme_to_evaluate)
                word2 = wiki2vec.get_word_vector(thema_value)
                similarity = utils.cosine_similarity(word1, word2)
            except KeyError:
                similarity = MINIMUM_COSINE_SIMILARITY
            if similarity > max_similarity:
                max_similarity = similarity
                best_match = thema_code
        if best_match is not None:
            print(theme_to_evaluate, "->", thema_dict[best_match]["CodeDescription"], max_similarity)
            theme_evaluation_wiki2vec[theme_to_evaluate] = best_match
        
    return theme_evaluation_wiki2vec

theme_evaluation_wiki2vec = align_lists_with_synonyms_wiki2vec(noeuds_thema_pretraites, wiki2vec)

sénégal -> sénégal 0.99999994
edition -> evolution 0.44893438
ours -> yupik 0.384791
maltraitance -> négligence 0.56435037
transhumanisme -> bioéthique 0.5027729
genre -> protozoaire 0.38188612
alaska -> alaska 1.0000001
biographie -> histoire 0.46998644
bolivie -> bolivie 1.0
yémen -> yémen 1.0
hippocampe -> dada 0.3293102
dragon -> fléchette 0.37385482
sida -> vaccination 0.5347764
pompier -> camping 0.3426182
iran -> iran 1.0000001
semaine -> pâques 0.40029237
finlande -> finlande 1.0000001
peintre -> cloisonnisme 0.5071018
papillon -> natation 0.3748513
souffrance -> négligence 0.48267052
nuit -> lune 0.42426005
guatemala -> guatemala 0.99999994
préhistoire -> préhistoire 1.0000001
astronomie -> astrophysique 0.6789408
révolte -> mutinerie 0.65357226
documentaire -> documentaires 0.6659119
oman -> oman 1.0000001
débarquement -> namsos 0.4566301
rugby -> cricket 0.5725878
poisson -> lune 0.35236487
roller -> bmx 0.55635524
licorne -> astrologie 0.36180797
tchécoslovaquie -> tchécosl

## doc2vec

In [109]:
import spacy
modele_nlp_spacy = spacy.load('fr_core_news_lg') # modele

In [111]:
noeuds_thema_cache = {}
for code, theme in thema_code_dict.items():
    # Preprocess the code description
    noeuds_thema_cache[code] = theme
    noeuds_thema_cache[code]["Modele_CodeDescription"] = modele_nlp_spacy(theme["CodeDescription"].lower())
    # noeuds_thema_cache[code]["Modele_CodeNotes"] = modele_nlp_spacy(theme["CodeNotes"].lower())

noeuds_thema_cache

{'A': {'CodeValue': 'A',
  'CodeDescription': 'arts',
  'CodeNotes': 'utilisez tous les codes a* pour les ouvrages spécialisés et généraux, qu’ils soient richement illustrés ou majoritairement textuel. préférez les codes wf* pour les ouvrages liés à un loisir ou un passe-temps, en le complétant au besoin par le(s) code(s) a*. utilisez au besoin tous les codes a* avec d’autres codes et qualificateurs, particulièrement avec les qualificateurs de style 6*, de lieux 1* et historiques 3*',
  'CodeParent': '',
  'IssueNumber': 1,
  'Modified': 1.4,
  'NLP_CodeDescription': arts,
  'NLP_CodeNotes': utilisez tous les codes a* pour les ouvrages spécialisés et généraux, qu’ils soient richement illustrés ou majoritairement textuel. préférez les codes wf* pour les ouvrages liés à un loisir ou un passe-temps, en le complétant au besoin par le(s) code(s) a*. utilisez au besoin tous les codes a* avec d’autres codes et qualificateurs, particulièrement avec les qualificateurs de style 6*, de lieux 1* e

In [112]:
def alignement_doc2vec(noeuds_thema, spacy_nlp):

    evaluations_positives = {}
    SIMILARITE_COSINE_MINIMALE = -1

    for theme_a_evaluer in themes_jeu_validation_dr.keys():
        similarite_max = SIMILARITE_COSINE_MINIMALE
        meilleure_correspondance = None
        theme_to_evaluate_nlp = spacy_nlp(theme_a_evaluer)
        for thema_code, thema_valeur in noeuds_thema.items():
            thema_valeur_modele = thema_valeur["Modele_CodeDescription"] # name of thema theme  
            similarite = theme_to_evaluate_nlp.similarity(thema_valeur_modele)
            if similarite > similarite_max:
                similarite_max = similarite
                meilleure_correspondance = thema_code
        if meilleure_correspondance is not None:
            print(theme_a_evaluer, "->", noeuds_thema[meilleure_correspondance]["CodeDescription"], similarite_max)
            evaluations_positives[theme_a_evaluer] = meilleure_correspondance
        
    return evaluations_positives


evaluations_positives_doc2vec = alignement_doc2vec(noeuds_thema_cache, modele_nlp_spacy) 

C:\Users\dre\AppData\Local\Temp\ipykernel_54596\3022673155.py:12: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarite = theme_to_evaluate_nlp.similarity(thema_valeur_modele)


balle, ballon -> fléchette 0.5571151654286275
animaux domestiques -> plantes sauvages et cueillette comestibles 0.7600863321665284
sénégal -> sénégal 1.0
livre-jeu -> techno-thriller 0.6226885881212054
atelier d'écriture -> logiciel d’illustration et de dessin 0.7637448133138999
edition -> evolution 0.6820361120421728
ours -> documentaires jeunesse : ours 0.5186113587089525
théâtre contemporain -> théâtre 0.9187365837867748
maltraitance -> violence domestique 0.7488391574613409
transhumanisme -> constructivisme 0.8150649623147851
genre -> fantasy humoristique 0.532994392672803
alaska -> alaska 1.0
biographie -> historiographie 0.7384783147127199
bolivie -> bolivie 1.0
conte détourné, le loup et les 7 cabris -> le tibre et ses affluents 0.7470662811876552
aventure et mystère -> cosmologie et univers 0.8536465099852968
conte détourné -> roman sentimental historique 0.6771777351524145
yémen -> yémen 1.0
hippocampe -> corte 0.4393069802783025
cow-boy -> saxe-anhalt 0.675274198619341
dragon

In [113]:
calculer_metriques(evaluations_positives_doc2vec)

len(theme evaluation) 60
len(themes_jeu_validation_dr) 60
VP 15.0
[('sénégal', '1HFDS', ['1HFDS', '1HFDS']),
 ('alaska', '1KBB-US-WPNA', ['1KBB-US-WPNA', '1KBB-US-WPNA']),
 ('bolivie', '1KLSL', ['1KLSL', '1KLSL']),
 ('yémen', '1FBXY', ['1FBXY', '1FBXY']),
 ('père noël', '5HPD', ['5HPD', 'N/A']),
 ('sida', 'MJCJ2', ['MJCJ2', 'MJCJ2']),
 ('iran', '1FBN', ['1FBN', '1FBN']),
 ('finlande', '1DNF', ['1DNF', '1DNF']),
 ('guatemala', '1KLCG', ['1KLCG', '1KLCG']),
 ('préhistoire', '3B', ['3B', '3B']),
 ('documentaire', 'ATFR', ['YN', 'ATFR']),
 ('oman', '1FBXM', ['1FBXM', '1FBXM']),
 ('aide humanitaire', 'JKSR', ['JKSR', 'JKSR']),
 ('tchécoslovaquie', '1QBDK', ['1QBDK', '1QBDK']),
 ('londres', '1DDU-GB-ESL', ['JPT', '1DDU-GB-ESLF', '1DDU-GB-ESL'])]
FP 45
['balle, ballon -> fléchette vs φ=sports / jeux de ballon | SXD vs φ=SF',
 'animaux domestiques -> plantes sauvages et cueillette comestibles vs '
 'φ=animaux domestiques et de compagnie | WNPB vs φ=WNG',
 'livre-jeu -> techno-thriller vs φ=liv